### 1_Science_Technology_(PCA)

### Import

In [67]:
import pandas as pd
import numpy as np 

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from pyclustertend import hopkins
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
from sklearn.metrics import silhouette_score

import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

plt.style.use('classic')
sns.set_style("white")

### Data Scraping Oecd

In [68]:
# import pandas as pd
# import pandasdmx as sdmx
# import pandas_datareader.data as web

# pd.set_option('display.max_rows', None)#print everything rows
# pd.set_option('display.max_columns', None)#print everything columns

# series = 'STIO_2016'
# source = 'oecd'
# start = '2006-01-01'
# end = '2016-01-01'

# oecd = sdmx.Request('OECD')
# params = dict(startPeriod=start, endPeriod=end)
# data_msg = oecd.data('STIO_2016', key='all', params=params)
# data = data_msg.data[0]
# df = sdmx.to_pandas(data).unstack() #turn data into a dataframe
# print(df)

In [69]:
#df.to_csv("science_technology.csv")

### Data Info

In [70]:
df = pd.read_csv("science_technology.csv")
df.head()

,COUNTRY,INDICATOR,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,ARG,ADTERPOP_XT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ARG,BE_AF_NC,9.80,9.90,11.80,12.10,16.50,23.39,27.41,32.94,NaN,NaN
2,ARG,BE_AF_PPP,7.29,6.42,6.44,6.05,7.11,8.78,8.87,9.15,NaN,NaN
3,ARG,BE_AF_PPPK,7.76,6.66,6.55,6.11,7.11,8.55,8.45,8.48,NaN,NaN
4,ARG,BE_AF_XGDP,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN


In [71]:
df=pd.melt(df,
           id_vars=["COUNTRY","INDICATOR"],
           value_vars=["2006","2007","2008","2009","2010","2011","2012","2013","2014","2015"],
           var_name="TIME",
           value_name="VALUE")
df.head(2)

,COUNTRY,INDICATOR,TIME,VALUE
0,ARG,ADTERPOP_XT,2006,NaN
1,ARG,BE_AF_NC,2006,9.8
